In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report 


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [8]:
# Synthetic classification data
X = np.random.rand(2000, 20).astype(np.float32)  # 20 features
y = np.random.randint(0, 3, size=(2000,))        # 3 classes

X_train, X_test = X[:1600], X[1600:]
y_train, y_test = y[:1600], y[1600:]

train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_dataset  = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


**PreActivation Residual Connection******

In [9]:
class PreActResidualBlock(nn.Module):
    def __init__(self, features):
        super().__init__()

        self.bn1 = nn.BatchNorm1d(features)
        self.fc1 = nn.Linear(features, features)

        self.bn2 = nn.BatchNorm1d(features)
        self.fc2 = nn.Linear(features, features)

        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x

        out = self.fc1(self.relu(self.bn1(x)))
        out = self.fc2(self.relu(self.bn2(out)))

        out = out + identity   # Residual connection
        return out


In [10]:
class ResidualNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()

        self.fc_in = nn.Linear(input_size, 128)

        self.res1 = PreActResidualBlock(128)
        self.res2 = PreActResidualBlock(128)

        self.fc_mid = nn.Linear(128, 64)
        self.fc_out = nn.Linear(64, num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc_in(x))
        x = self.res1(x)
        x = self.res2(x)
        x = self.relu(self.fc_mid(x))
        x = self.fc_out(x)
        return x


In [11]:
model = ResidualNN(input_size=20, num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
epochs = 80
train_losses = []

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")


Epoch [1/80] - Loss: 0.1807
Epoch [2/80] - Loss: 0.1461
Epoch [3/80] - Loss: 0.1306
Epoch [4/80] - Loss: 0.1150
Epoch [5/80] - Loss: 0.1043
Epoch [6/80] - Loss: 0.1068
Epoch [7/80] - Loss: 0.1221
Epoch [8/80] - Loss: 0.1066
Epoch [9/80] - Loss: 0.1027
Epoch [10/80] - Loss: 0.1197
Epoch [11/80] - Loss: 0.1207
Epoch [12/80] - Loss: 0.0804
Epoch [13/80] - Loss: 0.0709
Epoch [14/80] - Loss: 0.0793
Epoch [15/80] - Loss: 0.0740
Epoch [16/80] - Loss: 0.0906
Epoch [17/80] - Loss: 0.0750
Epoch [18/80] - Loss: 0.1458
Epoch [19/80] - Loss: 0.1314
Epoch [20/80] - Loss: 0.0895
Epoch [21/80] - Loss: 0.0673
Epoch [22/80] - Loss: 0.0857
Epoch [23/80] - Loss: 0.0738
Epoch [24/80] - Loss: 0.0469
Epoch [25/80] - Loss: 0.0486
Epoch [26/80] - Loss: 0.0587
Epoch [27/80] - Loss: 0.0825
Epoch [28/80] - Loss: 0.0709
Epoch [29/80] - Loss: 0.0631
Epoch [30/80] - Loss: 0.0615
Epoch [31/80] - Loss: 0.0762
Epoch [32/80] - Loss: 0.0815
Epoch [33/80] - Loss: 0.0840
Epoch [34/80] - Loss: 0.0792
Epoch [35/80] - Loss: 0

In [16]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)

        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.30      0.37      0.33       133
           1       0.37      0.25      0.30       150
           2       0.23      0.26      0.25       117

    accuracy                           0.29       400
   macro avg       0.30      0.30      0.29       400
weighted avg       0.31      0.29      0.29       400

